In [ ]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split as surprise_train_test_split
from surprise import accuracy

In [ ]:
class HybridRecommendationSystem:
    def __init__(self):
        self.svm_model = SVC(kernel='linear')
        self.svd_model = SVD()
        self.label_encoder = LabelEncoder()
        self.is_trained = False

    def load_data(self, customers_file, products_file, ratings_file):
        with open(customers_file, 'r') as f:
            self.customers_data = json.load(f)

        with open(products_file, 'r') as f:
            self.products_data = json.load(f)

        with open(ratings_file, 'r') as f:
            self.ratings_data = json.load(f)

    def preprocess_data(self):
        customers_df = pd.DataFrame(self.customers_data)
        products_df = pd.DataFrame(self.products_data)
        ratings_df = pd.DataFrame(self.ratings_data)
        customers_df.rename(columns={'Id': 'CustomerID'}, inplace=True)
        products_df.rename(columns={'Id': 'ProductID'}, inplace=True)
        user_item_df = pd.merge(ratings_df, customers_df, on='CustomerID')
        user_item_df = pd.merge(user_item_df, products_df, on='ProductID')

        # Encode categorical features like 'Age', 'Region', 'Category', 'genre'
        label_encoder = LabelEncoder()
        user_item_df['Age_encoded'] = label_encoder.fit_transform(user_item_df['Age'])
        user_item_df['Region_encoded'] = label_encoder.fit_transform(user_item_df['Region'])
        user_item_df['Category_encoded'] = label_encoder.fit_transform(user_item_df['Category'])
        user_item_df['genre_encoded'] = label_encoder.fit_transform(user_item_df['genre'])
        
        # Encode user and product IDs
        user_item_df['user_id_encoded'] = self.label_encoder.fit_transform(user_item_df['CustomerID'])
        user_item_df['product_id_encoded'] = self.label_encoder.fit_transform(user_item_df['ProductID'])

        # Split data into train and test sets for collaborative filtering
        X_train, X_test, y_train = user_item_df[['user_id_encoded', 'product_id_encoded']], \
                                   user_item_df['Rate'], \
                                   user_item_df['Rating']

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42)

        # Prepare data for surprise-based collaborative filtering
        reader = Reader(rating_scale=(1, 5))
        self.data = Dataset.load_from_df(user_item_df[['CustomerID', 'ProductID', 'Rating']], reader)
        self.trainset, self.testset = surprise_train_test_split(self.data, test_size=0.2, random_state=42)

    def train_model(self):
        # Train collaborative filtering models
        self.svm_model.fit(self.X_train, self.y_train)
        self.svd_model.fit(self.trainset)
        self.is_trained = True

    def hybrid_recommendation(self, user_id, product_id):
        if not self.is_trained:
            print("Model is not trained. Call train_model() first.")
            return

        # Predict using collaborative filtering
        cf_prediction = self.svm_model.predict([[self.label_encoder.transform([user_id])[0], self.label_encoder.transform([product_id])[0]]])

        # Predict using collaborative filtering (Surprise)
        svd_prediction = self.svd_model.predict(user_id, product_id)

        # Combine predictions (you can customize this logic)
        hybrid_prediction = (cf_prediction + svd_prediction.est) / 2

        return hybrid_prediction

    def evaluate_model(self):
        if not self.is_trained:
            print("Model is not trained. Call train_model() first.")
            return

        # Evaluate collaborative filtering model (SVM)
        y_pred_cf = self.svm_model.predict(self.X_test)

        accuracy_cf = accuracy_score(self.y_test, y_pred_cf)
        recall_cf = recall_score(self.y_test, y_pred_cf, average='weighted')
        f1_cf = f1_score(self.y_test, y_pred_cf, average='weighted')
        confusion_cf = confusion_matrix(self.y_test, y_pred_cf)

        print("Collaborative Filtering Metrics:")
        print("Accuracy:", accuracy_cf)
        print("Recall:", recall_cf)
        print("F1 Score:", f1_cf)
        print("Confusion Matrix:")
        print(confusion_cf)

        # Evaluate collaborative filtering model (Surprise SVD)
        svd_predictions = self.svd_model.test(self.testset)

        accuracy_svd = accuracy.rmse(svd_predictions)
        print("Surprise SVD RMSE:", accuracy_svd)


In [ ]:
# Example usage:
if __name__ == "__main__":
    hybrid_system = HybridRecommendationSystem()
    hybrid_system.load_data('customers.json', 'products.json', 'ratings.json')
    hybrid_system.preprocess_data()
    hybrid_system.train_model()

    # Example recommendation for user_id and product_id
    user_id = 'user123'
    product_id = 'product456'
    recommendation = hybrid_system.hybrid_recommendation(user_id, product_id)
    print("Hybrid Recommendation for User:", user_id, "Product:", product_id, "Rating:", recommendation)

    # Evaluate the hybrid recommendation system
    hybrid_system.evaluate_model()